# 第 1 節 – 聊天啟動 (Foundry Local)

此筆記本啟動 Foundry Local，下載首選模型別名，並執行標準和串流聊天完成。


# 情境
本次課程將介紹如何使用 Foundry Local 讓本地的小型語言模型進行回應的基本步驟。您將會：
- 安裝 SDK / 客戶端依賴項。
- 為選定的別名（預設為 `phi-4-mini`）初始化 Foundry Local 管理器。
- 應用防禦性猴子補丁，以容忍模型元數據中的可選字段。
- 發送標準聊天完成請求。
- 逐字元串流回應。

目標是驗證您的本地運行環境和網絡路徑，然後再進一步進行 RAG、路由或代理相關操作。


### 說明：依賴項安裝
安裝此簡易聊天流程所需的 Python 套件：
- `foundry-local-sdk`：管理本地模型和服務生命週期。
- `openai`：熟悉的客戶端抽象，用於聊天完成。
- `rich`：美觀的輸出，讓筆記本內容更清晰。

重新執行是安全的（具冪等性）。如果您的環境已經有這些套件，可以跳過。


In [1]:
# Install required libraries (idempotent)
%pip install -q foundry-local-sdk openai rich


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 解釋：核心匯入
引入整個筆記本中使用的模組：
- `FoundryLocalManager` 用於與本地模型運行時進行交互。
- `OpenAI` 客戶端，讓我們可以重用熟悉的聊天完成 API 介面。
- `rich.print` 用於樣式化輸出。

這裡不會進行任何網路呼叫——僅僅是準備命名空間。


In [2]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

### 說明：管理器初始化與元數據修補
初始化 `FoundryLocalManager` 以選定的別名，並應用防禦性猴子修補，以優雅地處理服務回應中 `promptTemplate` 可能為 `null` 的情況。

主要成果：
- 確認服務狀態和端點。
- 列出已緩存的模型（驗證本地存儲）。
- 解決別名的具體模型 ID（用於後續的聊天調用）。

如果在原始服務元數據中遇到驗證問題，此模式展示了如何在不分叉 SDK 的情況下進行清理。


In [3]:
# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:50262/v1

Cached models:
[
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-generic-gpu:4',
        version='4',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-generic-gpu/versions/4',
        file_size_mb=3809,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-generic-gpu:3',
        version='3',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-generic-gpu/versions/3',
        file_size_mb=700,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-generic-gpu:1',
        version='1',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-generic-gpu/versions/1',
        file_size_mb=2211,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    )
]

Using model id: Phi-4-mini-instruct-generic-gpu:4

### 解釋：基本聊天完成
建立一個與 `OpenAI` 相容的客戶端，指向本地 Foundry 端點，並執行一次非串流的聊天完成。重點如下：
- 確保模型回應無錯誤。
- 驗證延遲 / 輸出格式。
- 保持 `max_tokens` 適度以節省資源。

如果失敗，請重新檢查 Foundry Local 服務是否正在運行以及別名是否正確解析。


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':prompt}],
    max_tokens=120,
    temperature=0.5
)
print(resp.choices[0].message.content)

Local inference for privacy refers to the practice of performing data analysis on a local device without sending 
sensitive information to a central server. Two benefits of this approach are:


1. **Enhanced Privacy**: Local inference keeps personal data on the user's device, reducing the risk of data 
breaches and unauthorized access. Since the data is not transmitted over the network, it is less susceptible to 
interception by malicious actors.


2. **Data Sovereignty**: Users retain control over their data, as it does not leave their device. This means that 
individuals or organizations can comply with local data protection regulations, such as the General

### 說明：串流聊天完成
展示了令牌串流以改善感知延遲和互動式使用者體驗。迴圈會在增量變化到達時逐步打印：
- 適用於聊天介面，早期的部分輸出非常重要。
- 可用於測量令牌吞吐量與完整完成延遲之間的差異。

您可以調整此模式來累積令牌、更新進度小工具或在生成過程中中途中止。


In [5]:
# Streaming example
stream = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}],
    stream=True,
    max_tokens=60,
    temperature=0.4
)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

latency

and

bandwidth

usage

.

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵信息，建議使用專業人工翻譯。我們對因使用此翻譯而產生的任何誤解或誤釋不承擔責任。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
